In [1]:
import torch
import torch.utils.cpp_extension
from pathlib import Path
import random
import triton
import triton.language as tl

In [2]:
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [3]:
def test_correctness(softmax_impl, repeats=100):
    for i in range(repeats):
        row = random.randint(1, 1024 * 32)
        cols = random.randint(1, 1024 * 32)
        a = torch.randn((row, cols), device="cuda", dtype=torch.float32)
        torch.testing.assert_close(softmax_impl(a), torch.nn.functional.softmax(a, dim=-1), msg=f"row = {row}, cols = {cols}")
    return softmax_impl

In [ ]:
test_correctness(lambda a: torch.nn.functional.softmax(a, dim=-1))

In [5]:
def load_cuda_impl(file_name):
    cuda_src = Path(file_name).read_text()
    cpp_src = """
    torch::Tensor softmax(const torch::Tensor& x);
    """
    my = torch.utils.cpp_extension.load_inline(
        "my", cpp_src, cuda_src,
        functions=['softmax'], extra_cuda_cflags=['--ptxas-options=-v'], verbose=True,
    )
    return my.softmax

In [6]:
kernels = {}

In [7]:
def add_kernel(name, kernel):
    kernels[name] = kernel

In [8]:
add_kernel("Torch", torch.nn.functional.softmax)

## Naive

In [9]:
def naive_pytorch_softmax(x, dim=None):
    m = torch.max(x, dim=dim, keepdim=True)[0]
    e = torch.exp(x - m)
    s = torch.sum(e, dim=dim, keepdim=True)
    return e / s

In [10]:
add_kernel("Naive", test_correctness(lambda a: naive_pytorch_softmax(a, dim=-1)))

In [11]:
@torch.compile
def compiled_naive_pytorch_softmax(x, dim=None):
    return naive_pytorch_softmax(x, dim=dim)

In [12]:
add_kernel("CompiledNaive", test_correctness(lambda a: compiled_naive_pytorch_softmax(a, dim=-1)))

## CUDA

In [ ]:
softmax1 = add_kernel("CUDA1", test_correctness(load_cuda_impl("softmax1.cu")))

In [ ]:
softmax2 = add_kernel("CUDA2", test_correctness(load_cuda_impl("softmax2.cu")))

In [ ]:
softmax2_shfl = add_kernel("CUDA2_SHFL", test_correctness(load_cuda_impl("softmax2_shfl.cu")))

In [16]:
# softmax3 = add_kernel("CUDA3", test_correctness(load_cuda_impl("softmax3.cu")))

In [ ]:
softmax3_shfl = add_kernel("CUDA3_SHFL", test_correctness(load_cuda_impl("softmax3_shfl.cu")))

In [ ]:
softmax5 = add_kernel("CUDA5", test_correctness(load_cuda_impl("softmax5.cu")))

## Triton

In [19]:
@triton.jit
def triton_softmax1_kernel(
    x,
    x_row_stride,
    y,
    y_row_stride,
    n_cols,
    block_size: tl.constexpr,
):
    row_idx = tl.program_id(0)
    x_row_start = x + row_idx * x_row_stride
    col_offsets = tl.arange(0, block_size)
    row_ptrs = x_row_start + col_offsets
    mask = col_offsets < n_cols
    row = tl.load(row_ptrs, mask=mask, other=float('-inf'))

    row_max = tl.max(row, axis=0)
    safe_row = row - row_max
    exp_row = tl.exp(safe_row)
    sum_row = tl.sum(exp_row, axis=0)
    y_row_start = y + row_idx * y_row_stride
    y_ptrs = y_row_start + col_offsets
    tl.store(y_ptrs, exp_row / sum_row, mask=mask)

def triton_softmax1(x: torch.Tensor) -> torch.Tensor:
    x_flatten = x.reshape(-1, x.size(-1))
    y = torch.empty_like(x_flatten)
    rows, cols = x_flatten.shape
    block_size = triton.next_power_of_2(cols)
    num_warps = 4
    if block_size > 2047:
        num_warps = 8
    elif block_size > 4095:
        num_warps = 16

    grid = (rows, )
    triton_softmax1_kernel[grid](x_flatten, x_flatten.stride(0), y, y.stride(0), cols, block_size, num_warps=num_warps)
    
    return y.view_as(x)

add_kernel("Triton1", test_correctness(triton_softmax1))

In [20]:
@triton.jit
def triton_softmax2_kernel(
    x,
    x_row_stride,
    y,
    y_row_stride,
    n_cols,
    block_size: tl.constexpr,
):
    row_idx = tl.program_id(0)
    x_row_start = x + row_idx * x_row_stride
    y_row_start = y + row_idx * y_row_stride
    offsets = tl.arange(0, block_size)
    row_max = tl.full((), -float("inf"), dtype=tl.float32)
    for b in range(0, (n_cols + block_size - 1) // block_size):
        idx = b * block_size + offsets
        mask = idx < n_cols
        x_vals = tl.load(x_row_start + idx, mask=mask)
        row_max = tl.maximum(row_max, tl.max(x_vals.to(tl.float32)))

    row_sum = tl.zeros((), dtype=tl.float32)
    for b in range(0, (n_cols + block_size - 1) // block_size):
        idx = b * block_size + offsets
        mask = idx < n_cols
        x_vals = tl.load(x_row_start + idx, mask=mask, other=-float("inf"))
        x_vals = tl.exp(x_vals - row_max)
        row_sum += tl.sum(x_vals)
        tl.store(y_row_start + idx, x_vals, mask=mask)

    for b in range(0, (n_cols + block_size - 1) // block_size):
        idx = b * block_size + offsets
        mask = idx < n_cols
        y_vals = tl.load(y_row_start + idx, mask=mask)
        y_vals /= row_sum
        tl.store(y_row_start + idx, y_vals, mask=mask)
    

def triton_softmax2(x: torch.Tensor) -> torch.Tensor:
    x_flatten = x.reshape(-1, x.size(-1))
    y = torch.empty_like(x_flatten)
    rows, cols = x_flatten.shape
    block_size = 1024 * 32
    num_warps = 4
    if block_size > 2047:
        num_warps = 8
    elif block_size > 4095:
        num_warps = 16
    grid = (rows,)
    triton_softmax2_kernel[grid](x_flatten, x_flatten.stride(0), y, y.stride(0), cols, block_size, num_warps=num_warps)
    return y.view_as(x)

add_kernel("Triton2", test_correctness(triton_softmax2))

In [21]:
@triton.jit
def triton_softmax3_kernel(
    x,
    x_row_stride,
    y,
    y_row_stride,
    n_cols,
    block_size: tl.constexpr,
):
    row_idx = tl.program_id(0)
    x_row_start = x + row_idx * x_row_stride
    y_row_start = y + row_idx * y_row_stride
    current_max = float('-inf')
    current_sum = 0.0
    
    for block_offset in tl.range(0, n_cols, block_size):
        col_offsets = block_offset + tl.arange(0, block_size)
        block = tl.load(x_row_start + col_offsets, mask=col_offsets < n_cols, other=float('-inf'))
        new_max = tl.maximum(current_max, tl.max(block))
        current_sum *= tl.exp(current_max - new_max)
        current_sum += tl.sum(tl.exp(block - new_max))
        current_max = new_max

    for block_offset in tl.range(0, n_cols, block_size):
        col_offsets = block_offset + tl.arange(0, block_size)
        block = tl.load(x_row_start + col_offsets, mask=col_offsets < n_cols, other=float('-inf'))
        tl.store(y_row_start + col_offsets, tl.exp(block - current_max) / current_sum, mask=col_offsets < n_cols)


def triton_softmax3(x: torch.Tensor) -> torch.Tensor:
    x_flatten = x.reshape(-1, x.size(-1))
    y = torch.empty_like(x_flatten)
    rows, cols = x_flatten.shape
    block_size = 32 * 1024
    num_warps = 32
    grid = (rows, )
    triton_softmax3_kernel[grid](x_flatten, x_flatten.stride(0), y, y.stride(0), cols, block_size, num_warps=num_warps)
    return y.view_as(x)

add_kernel("Triton3", test_correctness(triton_softmax3))

## Benchmark

In [28]:
@triton.testing.perf_report(
    triton.testing.Benchmark(
        x_names=['size'],  # Argument names to use as an x-axis for the plot.
        x_vals=[2**i for i in range(12, 20, 1)],  # Different possible values for `x_name`.
        x_log=True,  # x axis is logarithmic.
        line_arg='provider',  # Argument name whose value corresponds to a different line in the plot.
        line_vals=list(kernels.keys()),  # Possible values for `line_arg`.
        line_names=list(kernels.keys()),  # Label name for the lines.
        # styles=[('blue', '-'), ('green', '-'), ('red', '-'), ('pink', '-'), ('yellow', '-'), ('black', '-'), ('orange', '-'), ('purple', '-'), ('cyan', '-'), ('gray', '-'), ('gray', '--')],  # Line styles.
        ylabel='GB/s',  # Label name for the y-axis.
        plot_name='softmax-performance',  # Name for the plot. Used also as a file name for saving the plot.
        args={},  # Values for function arguments not in `x_names` and `y_name`.
    ))
def triton_softmax_benchmark(size, provider):
    x = torch.rand((1, size), device="cuda", dtype=torch.float32)
    quantiles = [0.5, 0.2, 0.8]
    ms, min_ms, max_ms = triton.testing.do_bench(lambda: kernels[provider](x), quantiles=quantiles)
    gbps = lambda ms: 2 * x.numel() * x.element_size() * 1e-9 / (ms * 1e-3)
    return gbps(ms), gbps(max_ms), gbps(min_ms)

In [ ]:
triton_softmax_benchmark.run(print_data=True, show_plots=True)

In [24]:
@triton.testing.perf_report(
    triton.testing.Benchmark(
        x_names=['size'],  # Argument names to use as an x-axis for the plot.
        x_vals=[2**i for i in range(12, 30, 1)],  # Different possible values for `x_name`.
        x_log=True,  # x axis is logarithmic.
        line_arg='provider',  # Argument name whose value corresponds to a different line in the plot.
        line_vals=["CUDA5", "Naive"],  # Possible values for `line_arg`.
        line_names=["CUDA5", "Naive"],  # Label name for the lines.
        # styles=[('blue', '-'), ('green', '-'), ('red', '-'), ('pink', '-'), ('yellow', '-'), ('black', '-'), ('orange', '-'), ('purple', '-'), ('cyan', '-'), ('gray', '-'), ('gray', '--')],  # Line styles.
        ylabel='GB/s',  # Label name for the y-axis.
        plot_name='softmax-performance',  # Name for the plot. Used also as a file name for saving the plot.
        args={},  # Values for function arguments not in `x_names` and `y_name`.
    ))
def triton_softmax_benchmark(size, provider):
    x = torch.rand((1, size), device="cuda", dtype=torch.float32)
    quantiles = [0.5, 0.2, 0.8]
    ms, min_ms, max_ms = triton.testing.do_bench(lambda: kernels[provider](x), quantiles=quantiles)
    gbps = lambda ms: 2 * x.numel() * x.element_size() * 1e-9 / (ms * 1e-3)
    return gbps(ms), gbps(max_ms), gbps(min_ms)

In [ ]:
triton_softmax_benchmark.run(print_data=True, show_plots=True)